In [1]:
import pandas as pd

# READ CENSUS CODE FILE

In [3]:
fips = pd.read_excel("/Users/Livi/Projects/blm/capcookie/data/raw/all-geocodes-v2016.xlsx", header=4)

In [4]:
fips.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


In [5]:
fips = fips.iloc[:, slice(1,7)]

In [6]:
fips.columns = ['statecode','countycode','countysubcode','placecode','citycode','areaname']

In [7]:
fips.head()

,statecode,countycode,countysubcode,placecode,citycode,areaname
0,0,0,0,0,0,United States
1,1,0,0,0,0,Alabama
2,1,1,0,0,0,Autauga County
3,1,3,0,0,0,Baldwin County
4,1,5,0,0,0,Barbour County


# UPLOAD CSV WITH STATE ABBREVIATIONS

In [10]:
states = pd.read_csv('/Users/Livi/Projects/blm/capcookie/data/raw/states.csv',header=0)

In [11]:
states.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


# LEFT MERGE ABBREVIATION AND STATE CODE (my main df uses abbrevations)

In [12]:
statecode = pd.merge(states,fips,left_on='State',right_on='areaname')

In [13]:
statecode = statecode[['Code','statecode']].drop_duplicates().reset_index(drop=True)

In [14]:
statecode.tail()

,Code,statecode
46,VA,51
47,WA,53
48,WV,54
49,WI,55
50,WY,56


# UPLOAD KILLED-BY-POLICE DF

In [22]:
path ='/Users/Livi/Projects/blm/capcookie/data/'

In [23]:
df = pd.read_csv(path + "interim/fatalwrangling.csv", index_col=0)

In [25]:
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


# LEFT MERGE WITH STATE CODE (so I can compare with CENSUS CODE df)

In [26]:
df = pd.merge(df,statecode,left_on='state',right_on='Code', how='left')

In [27]:
df.tail()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,Code,statecode
4932,5998,Darius Washington,2020-07-18,shot,gun,24.000000,M,B,Chicago Heights,IL,False,attack,Foot,False,IL,17
4933,5999,Vincent Harris,2020-07-20,shot,gun,51.000000,M,B,Baton Rouge,LA,False,attack,Not fleeing,True,LA,22
4934,6003,TK TK,2020-07-21,shot,gun,36.538446,M,W,Oklahoma City,OK,False,other,Not fleeing,False,OK,40
4935,6004,Southern Jeremy,2020-07-21,shot,gun,22.000000,M,B,Sacramento,CA,False,attack,Not fleeing,True,CA,6
4936,6009,Christopher Poor,2020-07-25,shot,undetermined,49.000000,M,W,The Village,OK,False,undetermined,Other,False,OK,40


# MAKE CENSUS CODE DF ONLY INCLUDE ENTRIES THAT HAVE "PLACE CODES"
## "Cities" in my main df corresponds to areas with "place codes" in Census df

In [28]:
fipsplace = fips[fips.placecode != 0]

In [29]:
fipsplace.reset_index(inplace=True)

## Got rid of extraneous columns

In [30]:
amen = fipsplace[['statecode','placecode','areaname']]

In [31]:
amen.head()

,statecode,placecode,areaname
0,1,124,Abbeville city
1,1,460,Adamsville city
2,1,484,Addison town
3,1,676,Akron town
4,1,820,Alabaster city


In [32]:
fipsplace.areaname = fipsplace.areaname.str.title()

/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## There's a problem:
### The "city" name in df doesn't always perfectly match to the "areaname" in fipsplace (Census df)
### Fipsplace (census df) adds "Town" or "City" or "Village" to some names and it makes the match imperfect
### I'm trying to make cols for 1st word, 1st two words, or 1st three words in  the Census df
### I'll also make cols for main df: if city name is one word, I'll make a "first word" col in main df and match that to "first word" in Censusdf
### I'll do the same for Cities in main df that are two-words-long or three-words-long.

In [33]:
fipsplace['first_word'] = fipsplace.areaname.str.split(' ').apply(lambda x: x[0])
fipsplace['first_two_words'] = fipsplace.areaname.str.split(' ').apply(lambda x: "{} {}".format(x[0], x[1]) if len(x)>1 else None)
fipsplace['first_three_words'] = fipsplace.areaname.str.split(' ').apply(lambda x: "{} {} {}".format(x[0], x[1], x[2]) if len(x)>2 else None)
fipsplace.head()

/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

,index,statecode,countycode,countysubcode,placecode,citycode,areaname,first_word,first_two_words,first_three_words
0,69,1,0,0,124,0,Abbeville City,Abbeville,Abbeville City,None
1,70,1,0,0,460,0,Adamsville City,Adamsville,Adamsville City,None
2,71,1,0,0,484,0,Addison Town,Addison,Addison Town,None
3,72,1,0,0,676,0,Akron Town,Akron,Akron Town,None
4,73,1,0,0,820,0,Alabaster City,Alabaster,Alabaster City,None


##  New col in DF that has the Length of the city name

In [34]:
df['len_city'] = df.city.str.split(' ').apply(lambda x: len(x))

## Splitting DF into "shortdf" with 1-word citynames, "mediumdf" with 2-word citynames, and "longdf" with 3-or-more-word citynames

In [38]:
longdf = df[df.len_city > 2]
longdf['first_three_words'] = longdf.city.str.split(' ').apply(lambda x: "{} {} {}".format(x[0], x[1], x[2]))

mediumdf = df[df.len_city == 2]
mediumdf['first_two_words'] = mediumdf.city.str.split(' ').apply(lambda x: "{} {}".format(x[0], x[1]))

shortdf = df[df.len_city == 1]
shortdf['first_word'] = shortdf.city

/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/Livi/opt/anaconda3/envs/cap2/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


### Some cities in LONGDF are "North Los Angelos" or "North St. Louis" which are in the CENSUS DF (fipsplace) as "Los Angelos" and "St. Louis."
### I haven't yet checked if SHORTDF or MEDIUMDF have similar cities, but I will****

In [36]:
longdf[longdf.first_three_words.str.startswith("North" or "South" or "East" or "West")]

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,Code,statecode,len_city,first_three_words
23,45,Salvador Figueroa,2015-01-11,shot and Tasered,gun,29.0,M,H,North Las Vegas,NV,False,attack,Foot,False,NV,32,3,North Las Vegas
643,787,James Marcus Brown,2015-08-29,shot,gun,25.0,M,B,North Las Vegas,NV,False,attack,Foot,False,NV,32,3,North Las Vegas
1723,1989,Aaron Marquis Ballard,2016-10-24,shot,gun,19.0,M,B,North St. Louis,MO,False,attack,Car,False,MO,29,3,North St. Louis
2585,3000,George Randall Newman,2017-10-01,shot,gun,26.0,M,W,North Fort Collins,CO,False,attack,Not fleeing,True,CO,8,3,North Fort Collins
3541,4146,Gonzalo Rico-Jimenez,2018-10-31,shot,vehicle,26.0,M,H,North Las Vegas,NV,False,attack,Car,False,NV,32,3,North Las Vegas
4529,5349,Jamari Daiwon Tarver,2020-01-02,shot,vehicle,26.0,M,B,North Las Vegas,NV,False,attack,Car,True,NV,32,3,North Las Vegas
4751,5725,Yamil Acevedo,2020-04-06,shot,vehicle,29.0,M,H,North Miami Beach,FL,False,attack,Car,False,FL,12,3,North Miami Beach
4866,5906,Ruben Smith,2020-05-28,shot,gun,35.0,M,B,North Little Rock,AR,False,other,Not fleeing,False,AR,5,3,North Little Rock
4930,5995,TK TK,2020-07-16,shot,gun,60.0,M,W,North Fort Myers,FL,True,attack,Not fleeing,False,FL,12,3,North Fort Myers
